In [3]:
import openpyxl
from pathlib import Path
import os
from datetime import datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs


In [4]:
repodir = Path("../../") 
inputdir = repodir / "data" / "input-form"

In [5]:
os.listdir(inputdir)

['fireveg-trait-input-spreadsheet_V3_DE_trial.xlsx']

In [13]:
inputfile=inputdir / 'fireveg-trait-input-spreadsheet_V3_DE_trial.xlsx'
wb = openpyxl.load_workbook(inputfile,data_only=True)
wb.worksheets

[<Worksheet "Instructions">,
 <Worksheet "Input table">,
 <Worksheet "traits">,
 <Worksheet "references">,
 <Worksheet "species codes">,
 <Worksheet "vocabularies">]

### References table

In [78]:
ws=wb["references"]

In [82]:
for table in ws.tables:
    print(table)

list_refs


In [83]:
table = ws.tables['list_refs']

In [86]:
tbl=ws[table.ref]

In [94]:
for cell in tbl[0]:
    print(cell.value)

ref_number
ref_code
ref_info
DOI
original_ref_code
original_ref_info


In [103]:
for cell in tbl[len(tbl)-1]:
    print(cell.value)

272
Sheringham 2022
Sheringham, P. 2022. Post Fire Response of Plant species in the Gibraltar Range.  Unpublished report 
None
None
None


### Input table

In [14]:
ws=wb["Input table"]

In [15]:
ws['A1':'B7']

((<Cell 'Input table'.A1>, <Cell 'Input table'.B1>),
 (<Cell 'Input table'.A2>, <Cell 'Input table'.B2>),
 (<Cell 'Input table'.A3>, <Cell 'Input table'.B3>),
 (<Cell 'Input table'.A4>, <Cell 'Input table'.B4>),
 (<Cell 'Input table'.A5>, <Cell 'Input table'.B5>),
 (<Cell 'Input table'.A6>, <Cell 'Input table'.B6>),
 (<Cell 'Input table'.A7>, <Cell 'Input table'.B7>))

In [25]:
for table in ws.tables:
    print(table)
    table.values()

input_table


AttributeError: 'str' object has no attribute 'values'

In [28]:
table = ws.tables['input_table']

In [29]:
for row in table:
    print(row)

('id', '24')
('name', 'input_table')
('displayName', 'input_table')
('ref', 'A1:O46')
('headerRowCount', '1')
('totalsRowShown', '0')
('headerRowDxfId', '36')
('dataDxfId', '34')
('headerRowBorderDxfId', '35')
('tableBorderDxfId', '33')


In [38]:
table.ref

'A1:O46'

In [37]:
for row in ws[table.ref]:
    for cell in row:
        print(cell.value)

main_source
original_sources
original_species_name
species_code
species name
Trait code
Trait name
Trait type
raw_value
norm_value
Best
Lower
Upper
method_of_estimation
notes
MacKenzie Auld Keith Ooi 2021 
None
Boronia anemonifolia subsp. anemonifolia
12257
Boronia anemonifolia subsp. anemonifolia
surv1
Resprouting - full canopy scorch
categorical
None
Most
None
None
None
Direct observation (many)
None
MacKenzie Auld Keith Ooi 2021 
None
Boronia floribunda
5740
Boronia floribunda
surv1
Resprouting - full canopy scorch
categorical
None
Most
None
None
None
Direct observation (many)
None
MacKenzie Auld Keith Ooi 2021 
None
Boronia fraseri
5741
Boronia fraseri
surv1
Resprouting - full canopy scorch
categorical
None
None
None
None
None
Direct observation (many)
None
MacKenzie Auld Keith Ooi 2021 
None
Boronia ledifolia
5744
Boronia ledifolia
surv1
Resprouting - full canopy scorch
categorical
None
None
None
None
None
Direct observation (many)
None
MacKenzie Auld Keith Ooi 2021 
None
Boronia 

In [46]:
dir(table)

['__add__',
 '__attrs__',
 '__class__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__elements__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__namespaced__',
 '__ne__',
 '__nested__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_initialise_columns',
 '_path',
 '_rel_id',
 '_rel_type',
 '_write',
 'autoFilter',
 'column_names',
 'comment',
 'connectionId',
 'dataCellStyle',
 'dataDxfId',
 'displayName',
 'extLst',
 'from_tree',
 'headerRowBorderDxfId',
 'headerRowCellStyle',
 'headerRowCount',
 'headerRowDxfId',
 'id',
 'idx_base',
 'insertRow',
 'insertRowShift',
 'mime_type',
 'name',
 'namespace',
 'path',
 'published',
 'ref',
 'sortState',
 'tableBorderDxfId',
 'tableColumns',
 'tableStyleInfo',
 'tableType',
 'tagname',
 'to_tree',
 'tota

In [47]:
table.column_names

['main_source',
 'original_sources',
 'original_species_name',
 'species_code',
 'species name',
 'Trait code',
 'Trait name',
 'Trait type',
 'raw_value',
 'norm_value',
 'Best',
 'Lower',
 'Upper',
 'method_of_estimation',
 'notes']

In [48]:
table.headerRowCount

1

In [75]:
tbl=ws[table.ref]
hdr=table.column_names
surv1=list()
surv4=list()
records=list()
for k in range(1,len(tbl)):
    record=dict()
    for j in range(0,len(hdr)):
        record[hdr[j]]=tbl[k][j].value
    if record['Trait code']=='surv1':
        surv1.append(record)
    elif record['Trait code']=='surv4':
        surv4.append(record)
    else: 
        records.append(record)

In [77]:
surv1

[{'main_source': 'MacKenzie Auld Keith Ooi 2021 ',
  'original_sources': None,
  'original_species_name': 'Boronia anemonifolia subsp. anemonifolia',
  'species_code': 12257,
  'species name': 'Boronia anemonifolia subsp. anemonifolia',
  'Trait code': 'surv1',
  'Trait name': 'Resprouting - full canopy scorch',
  'Trait type': 'categorical',
  'raw_value': None,
  'norm_value': 'Most',
  'Best': None,
  'Lower': None,
  'Upper': None,
  'method_of_estimation': 'Direct observation (many)',
  'notes': None},
 {'main_source': 'MacKenzie Auld Keith Ooi 2021 ',
  'original_sources': None,
  'original_species_name': 'Boronia floribunda',
  'species_code': 5740,
  'species name': 'Boronia floribunda',
  'Trait code': 'surv1',
  'Trait name': 'Resprouting - full canopy scorch',
  'Trait type': 'categorical',
  'raw_value': None,
  'norm_value': 'Most',
  'Best': None,
  'Lower': None,
  'Upper': None,
  'method_of_estimation': 'Direct observation (many)',
  'notes': None},
 {'main_source': 'M

In [74]:
hdr 'surv1'

False